# Practise notebook for v1 of one-pass algorithms 

This notebook goes through the use of the one-pass algorithms (opa) package that has been developed for Destination Earth. Currently the package works for the following statistics: 

- mean 
- variance 
- standard deviation 
- minimum 
- maximum 
- threshold exceedance 

The two remaining statistics to be implemented are histograms and percentiles. 

First required modules are imported. This notebook is currently using data from nextGEMS re-gridded with the regridding tool devleoped by AQUA, working on Levante. This will ultimately change to data from the GSV interface. 

__If you are not working on Levante and want to use other test data there is no need to import the aqua reader packages__ 

Any xr.DataSet or xr.DataArray data can be used as long as it is on a lat / lon grid. 

In [13]:
import os
import xarray as xr 
import dask as dk
import numpy as np 

from one_pass.opa import *
from one_pass.opa import Opa # from module (file.py) import class name 

### remove if necessary ### 

os.chdir('/home/b/b382291/git/AQUA') # CHANGE TO CORRECT PATH OR REMOVE IF USING OTHER DATA 
from aqua import Reader
from aqua.reader import catalogue


Now load data from the regridder. This is just to have test data loaded into memory and can be any climate data in xr.DataSet or xr.Array format. 

__NO NEED TO RUN THIS SECTION IF YOU ARE USING OTHER TEST DATA__ 

Here we will keep data (dataSet) and tas (dataArray) for demonstration

In [14]:
#catalogue();

#reader = Reader(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d", regrid="r020")
reader = Reader(model="ICON", exp="ngc2009", source="atm_2d_ml_R02B09", regrid="r020")

data_all = reader.retrieve(fix=False)
data = reader.regrid(data_all) # keeping variable set as at DataSet 
tas = data.tas
#tas = reader.regrid(data["tas"][:,:]) # also extracting dataArray of tas 


This is the path to the config file. In the config file you need to set time step of the data in minutes. This is the known timestep of your data (e.g 30 minutes, 60 minutes) which will be known from the call to the GSV interface (or simply look at dt in the time dimension of whatever data you're using). You also need to specifiy in here the file path of where you want to save data.

In [16]:
file_path = "/home/b/b382291/git/one_pass/config.yml"

## How to initalise the statistic

### Example 1 

Every statistic needs to be initalised with the paramaters of choice. Input parameters are: 

- __statistic__: the statistic that you want to calculate.
- Options: "mean", "var", "std", "min", "max", "threshExceed" 

- __statFreq__:  the time period over which you want to calculate this statistic.
- Options: "hourly", "3hourly", "6hourly", "daily", "weekly", "monthly", "annually" (currently do not have 'inifite' but will implement)

- __outputFreq__: the frequency in which you want the algorithm to output the data, e.g. putting outputFreq = daily with statFreq = hourly will output a dataSet with hourly data concatenated to a full day. Note this value can not be less than statFreq.
- Options: "hourly", "3hourly", "6hourly", "daily", "weekly", "monthly", "annually"

- __save__: do you want to save the output to netCDF. File path specified in the config file. Will save at the same frequency as the outputFreq.
- Options: True, False 

- __variable__: Currently the algorithm can only process one climate varaible. If you input a full dataSet with many variables, you must specify the data variable you wish to calculate the statistic on. This input is not required if you provide the algorithm a dataArray
- Options: any climate variable... 

- __threshold__:If you are calculating threshold exceedance, must provide a threshold. No input required for other statstics.
- Options: any floating point value or integer

- __configPath__: path to configuration file given above





In [31]:
daily_mean = Opa(statistic = "mean", stat_freq = "daily", output_freq = "daily", save = False, variable = "tas", config_path = file_path)

### Simulating streaming 

As the data is not currently streamed, streaming is being simulated simply by taking time slices through the data. Here we feed the algorithm new data as single time slices that span a full day (time step of the data is 30 minutes). The call to calculate the statistic is 

    > .compute 

Here we gave the function the full data set and specified the variable tas. After a full day of information has been provided, dm is returned as a completed dataSet containing the daily mean.

In [32]:
for i in range(0, 48, 1): 

    ds = data.isel(time=slice(i,i+1)) # extract moving window of tas variable
    dm = daily_mean.compute(ds)

dm


<xarray.Dataset>
Dimensions:  (time: 1, lat: 900, lon: 1800)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
Data variables:
    tas      (time, lat, lon) float64 dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
Attributes: (12/13)
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 152214
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    ...                   ...
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9
    regridded:            1
    OPA:                  daily mean calculated using one-pass algorithm

### Example 2 

Next we look at standard deviation and ask for an output Frequency of a week, while still requesting daily calculations. This means that we have to give the statistic a full week of data before it will return a completed output. Now we can see that the output dataSet has 7 days of daily standard deviations. 

In [19]:
weekly_std = Opa(statistic = "std", stat_freq = "daily", output_freq = "weekly", save = False, variable = "tas", config_path = file_path)

for i in range(0, 7*48, 1): 

    ds = data.isel(time=slice(i,i+1)) # extract moving window of tas variable
    dm = weekly_std.compute(ds)

dm

<xarray.Dataset>
Dimensions:  (time: 7, lat: 900, lon: 1800)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20 2020-01-21 ... 2020-01-26
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
Data variables:
    tas      (time, lat, lon) float64 dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
Attributes: (12/13)
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 152214
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    ...                   ...
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9
    regridded:            1
    OPA:                  daily std calculated using one-pass algorithm

### Example 3 
  
We now ask for the minimum value every three hours, output as a full day and to save the output to netCDF. Here we have removed the input variable, so need to give the data in dataArray format otherwise it won't work. For the statistics of min and max, the output dataSet will have two variables, the value of the min / max and another array called 'timings'. This array corresponds to the time at which the min/max occured. 'Finish saving' will print when the statistic saves successfully. 

In [20]:
three_hourly_min = Opa(statistic = "min", stat_freq = "3hourly", output_freq = "daily", save = True, config_path = file_path)

for i in range(0, 48, 1): 

    ds = tas.isel(time=slice(i,i+1)) # extract moving window of tas variable
    dm = three_hourly_min.compute(ds)

dm

finished saving


<xarray.Dataset>
Dimensions:  (time: 8, lat: 900, lon: 1800)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20T02:00:00 ... 2020-01-20T23:00:00
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
Data variables:
    tas      (time, lat, lon) float64 dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
    timings  (time, lat, lon) datetime64[ns] dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
Attributes:
    CDI_grid_type:                unstructured
    cell_methods:                 t: point
    height:                       2.0 m
    long_name:                    temperature in 2m
    number_of_grid_in_reference:  1
    param:                        0.0.0
    standard_name:                tas
    units:                        K
    OPA:                          3hourly min calculated using one-pass algor...

## A note on when the output will be given 

### Example 4 

In order for the algorithm to provide the final dataSet (here called dm), the algorithm needs to be given the correct number of time slices (GRIB messages). For example, below we ask for a three hourly variance, also output every 3 hours. As we have a timestep of 30 minutes, 6 GRIB messages are required to complete the statistic (180 minutes / 30 minutes). If not enough messages given, dm will be 'NoneType' until all the data is given. It is possible however to check how far through the statistic the algorithm is by using some of the attributes: 

- threeHourlyVar.count = 5 (number of GRIB messages fed into the statistic)
- threeHourlyVar.nData = 6 (number of GRIB messages required to complete the statstic)
- threeHourlyVar.minCum = cumulative minimum over these the values already given 

In [21]:
three_hourly_var = Opa(statistic = "var", stat_freq = "3hourly", output_freq = "3hourly", save = False, config_path = file_path)

for i in range(0, 5, 1): 

    ds = tas.isel(time=slice(i,i+1)) # extract moving window of tas variable
    dm = three_hourly_var.compute(ds)

dm

In [23]:
three_hourly_var.n_data

6

In [24]:
three_hourly_var.var_cum

dask.array<add, shape=(1, 900, 1800), dtype=float64, chunksize=(1, 900, 1800), chunktype=numpy.ndarray>

### Example 5 

It is possible that the more than one time slice (GRIB message) can be stored in memory. If this is the case, multiple time slices can be fed into the statistic. In the example below, 5 GRIB messages are being given to the algorithm in for each loop iteration. To fill this daily output we require nData = 24*60/30 = 48, so 48 GRIB messages exactly. However with this loop construciton we provide 50, so 2 more time slices than the algorithm requires. In this case, the statistic will fill, output dm, and start calculating the daily maximum with the two new GRIB messages provided. 

threeHourlyMax.count = 2 at the end of the loop below. 
threeHourlyMax.maxCum = maximum values of the two new timeslices given for the next day 

__dm will also be reset when the new time slices are given__ this is subject to change based on feedback from the user 

In [25]:
three_hourly_max = Opa(statistic = "max", stat_freq = "daily", output_freq = "daily", save = True, config_path = file_path)

for i in range(0, 48, 5): 

    ds = tas.isel(time=slice(i,i+5)) # extract moving window of tas variable
    dm = three_hourly_max.compute(ds)

dm

finished saving


<xarray.Dataset>
Dimensions:  (time: 1, lat: 900, lon: 1800)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
Data variables:
    tas      (time, lat, lon) float64 dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
    timings  (time, lat, lon) datetime64[ns] dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
Attributes:
    CDI_grid_type:                unstructured
    cell_methods:                 t: point
    height:                       2.0 m
    long_name:                    temperature in 2m
    number_of_grid_in_reference:  1
    param:                        0.0.0
    standard_name:                tas
    units:                        K
    OPA:                          daily max calculated using one-pass algorithm

### Example 6

It is possible to give the algorithm more data than required for the whole statstic. In the case below, we are giving the statistic 13 time slices when it only requires 12 to complete the 6 hour statistic. Again, this is not a problem, dm will be provided and the new statistic will start to compute. We can see that the new statistic has been given 1 extra time slice by running: 

threshExceed6hourly.count 

Currently threshold exceedance does not have an associated time array, it will only provide the number of times the threshold was exceeded. Would an associated timing array be useful?  

In [28]:
# initalising the statistic 
thresh_exceed_6_hourly = Opa(statistic = "threshExceed", stat_freq = "6hourly", output_freq = "6hourly", 
            save = False, threshold = 250, config_path = file_path)

for i in range(0, 12, 13): 

    ds = tas.isel(time=slice(i,i+13)) # extract moving window of tas variable
    dm = thresh_exceed_6_hourly.compute(ds)

dm

In [29]:
thresh_exceed_6_hourly.count

0